In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import make_classification

# Load the training and test data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Preprocess the training data
train_two = []
for i in range(train_data.shape[0]):
    z = len(train_data.iloc[i][0])
    x = 0
    for j in range(z):
        y = (ord(train_data.iloc[i][0][j]) - 65)
        if y < 0 or y > 25:
            x = 1
            break
    if x == 0:
        train_two.append(train_data.iloc[i])

train_two = pd.DataFrame(train_two)
train_data = train_two

train_label = train_data['Label']
train_feature20 = []

for i in range(train_data.shape[0]):
    x = [0] * 26
    z = len(train_data.iloc[i][0])
    for j in range(z):
        y = (ord(train_data.iloc[i][0][j]) - 65)
        x[y] += 1

    for j in range(26):
        x[j] /= z

    train_feature20.append(x)

train_feature20 = pd.DataFrame(train_feature20)

# Preprocess the test data
test_feature20 = []

for i in range(test_data.shape[0]):
    x = [0] * 26
    z = len(test_data.iloc[i][1])
    for j in range(z):
        y = (ord(test_data.iloc[i][1][j]) - 65)
        x[y] += 1

    for j in range(26):
        x[j] /= z

    test_feature20.append(x)

test_feature20 = pd.DataFrame(test_feature20)

# Create the StackingClassifier
estimators = [
    ('Random Forest', RandomForestClassifier(max_depth=60, random_state=83, max_features='sqrt', n_estimators=4000))
]

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=RandomForestClassifier())

# Feature selection
k = 10  # Replace with your desired value
selector = SelectKBest(f_classif, k=k)
train_feature20_selected = selector.fit_transform(train_feature20, train_label)
test_feature20_selected = selector.transform(test_feature20)

# Fit the stacking classifier to the selected training features
stacking_classifier.fit(train_feature20_selected, train_label)

# Predict labels for the test data
y_pred = stacking_classifier.predict(test_feature20_selected)

# Save predictions to CSV file
df = pd.DataFrame({'ID': range(501, 811), 'Label': y_pred})
df.to_csv('result4.csv', index=False)


c:\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 1  9 14 20 23 25] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
